In [1]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

import re

import pandas as pd

import nltk

import matplotlib.pyplot as plt

## Get and preprocess the data

In [2]:
df = pd.read_feather("../data/twitter_mutlimodal_hate_speech/data.feather")

In [3]:
labels = df["labels_str"].apply(pd.Series)

display(labels.head(5))
df.loc[16833, "tweet_text"]

# df[labels.columns] =  labels


display(pd.concat([labels[col] for col in labels.columns]).value_counts())


# One-hot encoded categories
# pd.get_dummies(labels)
oh_labels = pd.concat([
    pd.get_dummies(labels[li])
    for li in labels.columns
]).groupby(level=0).max().astype(bool)

df[oh_labels.columns] = oh_labels

display(df.head())

,0,1,2,3,4
0,Religion,Racist,Homophobe,NaN,NaN
1,OtherHate,OtherHate,OtherHate,NaN,NaN
2,NotHate,NotHate,NotHate,NaN,NaN
3,Racist,NotHate,NotHate,NaN,NaN
4,Racist,NotHate,Racist,NaN,NaN


NotHate      312039
Racist        63543
OtherHate     31548
Sexist        22805
Homophobe     16932
Religion       2607
dtype: int64

,img_url,labels,tweet_url,tweet_text,labels_str,tweet_id,img_text,Homophobe,NotHate,OtherHate,Racist,Religion,Sexist
0,http://pbs.twimg.com/tweet_video_thumb/D3gi9MH...,"[4, 1, 3]",https://twitter.com/user/status/11146793537140...,@FriskDontMiss Nigga https://t.co/cAsaLWEpue,"[Religion, Racist, Homophobe]",1114679353714016256,#YOUNGERU SAVE IT,True,False,False,True,True,False
1,http://pbs.twimg.com/ext_tw_video_thumb/106301...,"[5, 5, 5]",https://twitter.com/user/status/10630200488166...,My horses are retarded https://t.co/HYhqc6d5WN,"[OtherHate, OtherHate, OtherHate]",1063020048816660480,None,False,False,True,False,False,False
2,http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg,"[0, 0, 0]",https://twitter.com/user/status/11089273680753...,“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL S...,"[NotHate, NotHate, NotHate]",1108927368075374593,None,False,True,False,False,False,False
3,http://pbs.twimg.com/ext_tw_video_thumb/111401...,"[1, 0, 0]",https://twitter.com/user/status/11145585346356...,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,"[Racist, NotHate, NotHate]",1114558534635618305,None,False,True,False,True,False,False
4,http://pbs.twimg.com/media/Dl30pGIU8AAVGxO.jpg,"[1, 0, 1]",https://twitter.com/user/status/10352524802155...,“EVERYbody calling you Nigger now!” https://t....,"[Racist, NotHate, Racist]",1035252480215592966,None,False,True,False,True,False,False


In [6]:
# Homophobe = df.loc[df["Homophobe"], "tweet_text"]
# NotHate = df.loc[df["NotHate"], "tweet_text"]
# OtherHate = df.loc[df["OtherHate"], "tweet_text"]
# Racist = df.loc[df["Racist"], "tweet_text"]
# Religion = df.loc[df["Religion"], "tweet_text"]
# Sexist = df.loc[df["Sexist"], "tweet_text"]

### Cleaning text

In [4]:
def clean_tweets(vTEXT):
    """
    Removes URLs, tags and more from tweets. Extend it if needed.
    
    short words: \b\w{1,2}\b
    tags starting with @: @\w*
    URL: (https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b
    """
    vTEXT = re.sub(r'\b\w{1,2}\b|@\w*|(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

# Racist = Racist.apply(remove_urls)

In [5]:
data = df.copy()

data["text"] = data["tweet_text"].apply(clean_tweets)
data = data[["text", "Homophobe", "NotHate", "OtherHate", "Racist", "Religion", "Sexist"]]

# data = data[["text", "Homophobe", "NotHate", "OtherHate", "Racist", "Religion", "Sexist"]].melt(
# #     id_vars="tweet_text"
#     id_vars=['text'], value_vars=[
#         "Homophobe",
#         "NotHate",
#         "OtherHate",
#         "Racist",
#         "Religion",
#         "Sexist"
#     ],
#     var_name="category"
# )

# data = data[data["value"] & (data["category"] != "NotHate")].drop(columns=("value"))

In [6]:
data

,text,Homophobe,NotHate,OtherHate,Racist,Religion,Sexist
0,Nigga,True,False,False,True,True,False
1,horses are retarded,False,False,True,False,False,False
2,“NIGGA MOMMA YOUNGBOY SPITTING REAL SHIT NI...,False,True,False,False,False,False
3,xxSuGVNGxx: ran into this HOLY NIGGA TODAY 😭...,False,True,False,True,False,False
4,“EVERYbody calling you Nigger now!”,False,True,False,True,False,False
...,...,...,...,...,...,...,...
149818,would just say hes Donny the retard,False,True,True,False,False,True
149819,congrats nigga keep grinding,False,True,False,False,False,False
149820,nigga big shitty,False,True,False,True,False,False
149821,did she just say “ nigga” Rich? &amp; she sai...,False,True,False,True,False,False


In [10]:
# Here we only export the dataset to be able to use it in other notebooks (not needed to do again, as we have already exported 
# our dataset)
# data.to_excel('../data/twitter_mutlimodal_hate_speech/twitter_mutlimodal_hate_speech.xlsx')

### Stemming

In [7]:
# Load SnowballStemmer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenize_and_stem(text):
    # First tokenize by sentence, then by word to ensure that punctuation is caught as it's own token.
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # Filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation).
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # First tokenize by sentence, then by word to ensure that punctuation is caught as it's own token.
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # Filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation).
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [8]:
tweets_text = data["text"].tolist()

In [9]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in tweets_text:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

# Create a pandas DataFrame
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

print("There are '{}' items in our data frame.".format(str(vocab_frame.shape[0])))
print("Data frame contents: \n{}".format(vocab_frame.head(10)))

There are '1146606' items in our data frame.
Data frame contents: 
             words
nigga        nigga
hors        horses
are            are
retard    retarded
nigga        nigga
momma        momma
youngboy  youngboy
spit      spitting
real          real
shit          shit


## TF-IDF Encoding

In [10]:
# Define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(
                        max_df=0.8, 
                        max_features=200000,
#                         min_df=0.2, 
                        min_df=0.0015,
#                         min_df=0.002,
                        stop_words='english', 
                        use_idf=True, 
                        tokenizer=tokenize_and_stem, 
                        ngram_range=(1,3))

# Fit the vectorizer to synopses texts
tfidf_matrix = tfidf_vectorizer.fit_transform(tweets_text) 

print("TF-IDF matrix shape: {}".format(tfidf_matrix.shape))

C:\Users\User\anaconda3\envs\nlp-course-fri\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


TF-IDF matrix shape: (149823, 589)


In [11]:
print("TF-IDF vectors (each column is a document):\n{}\nRows:\n{}".format(tfidf_matrix.T.A, tfidf_vectorizer.get_feature_names()))

TF-IDF vectors (each column is a document):
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Rows:
['absolut', 'account', 'act', 'actual', 'ago', 'ain', 'aint', 'alreadi', 'alway', 'america', 'amp', 'ani', 'anim', 'anoth', 'anoth nigga', 'anyon', 'anyth', 'asf', 'asian', 'ask', 'ass', 'ass bitch', 'ass bitch fuck', 'ass nigga', 'away', 'aye', 'babe', 'babi', 'bad', 'bag', 'ball', 'bar', 'beat', 'becaus', 'befor', 'believ', 'best', 'bet', 'better', 'big', 'big dyke', 'big dyke energi', 'bint', 'birthday', 'birthday nigga', 'bit', 'bitch', 'bitch ass', 'bitch ass nigga', 'bitch fuck', 'bitch fuck bout', 'bitch nigga', 'black', 'bless', 'block', 'blond', 'bodi', 'book', 'border', 'bout', 'bout nigga', 'boy', 'break', 'bring', 'bro', 'broke', 'broke nigga', 'brother', 'bruh', 'buildthewal', 'buildthewal maga', 'busi', 'buy', 'came', 'car', 'card', 'care', 'cat', 'catch', 'caus', 'chan

In [12]:
terms = tfidf_vectorizer.get_feature_names()

# from sklearn.metrics.pairwise import cosine_similarity
# dist = 1 - cosine_similarity(tfidf_matrix)

## Clustering

In [13]:
num_clusters = 10
km = KMeans(n_clusters=num_clusters)

# Perform clustering
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()
print("Clusters: {}".format(clusters))

Clusters: [4, 0, 1, 1, 1, 1, 2, 6, 3, 4, 1, 1, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4, 8, 7, 7, 7, 1, 5, 1, 6, 3, 1, 1, 3, 1, 5, 7, 1, 1, 6, 1, 7, 1, 1, 1, 1, 1, 1, 0, 7, 1, 1, 1, 1, 1, 5, 5, 0, 1, 7, 7, 1, 5, 5, 1, 0, 1, 8, 6, 7, 1, 1, 1, 1, 0, 8, 6, 3, 7, 1, 1, 1, 1, 1, 1, 4, 1, 6, 7, 1, 0, 6, 1, 1, 7, 1, 7, 4, 1, 1, 5, 1, 1, 3, 1, 1, 3, 1, 0, 1, 1, 1, 4, 8, 8, 1, 1, 6, 7, 1, 1, 4, 4, 1, 0, 1, 1, 1, 0, 5, 1, 1, 7, 0, 5, 4, 1, 1, 6, 0, 6, 1, 6, 8, 9, 5, 1, 1, 1, 1, 1, 5, 1, 4, 5, 1, 8, 1, 1, 7, 1, 1, 1, 1, 4, 1, 3, 4, 5, 8, 7, 5, 5, 1, 1, 1, 1, 1, 0, 1, 0, 5, 1, 2, 1, 1, 6, 1, 1, 8, 1, 1, 0, 7, 5, 6, 1, 6, 3, 1, 1, 1, 0, 1, 0, 6, 1, 1, 9, 1, 1, 9, 1, 1, 4, 4, 3, 1, 0, 1, 1, 1, 4, 6, 1, 1, 1, 1, 1, 1, 3, 1, 8, 1, 1, 1, 1, 8, 4, 1, 1, 1, 6, 1, 5, 3, 6, 1, 8, 4, 1, 1, 1, 0, 5, 1, 1, 1, 9, 6, 1, 1, 5, 1, 1, 1, 6, 1, 1, 1, 1, 1, 7, 4, 1, 9, 4, 0, 1, 6, 8, 1, 6, 8, 1, 1, 1, 1, 0, 1, 0, 1, 7, 1, 6, 1, 1, 1, 1, 1, 7, 0, 4, 1, 7, 1, 0, 7, 4, 1, 1, 7, 1, 3, 1, 4, 1, 1, 7, 6, 0, 1, 1, 6, 1, 8, 1, 4, 1,

In [17]:
# # Elbow method
# distortions = []
# for i in range(1, 20):
#     km = KMeans(
#         n_clusters=i, init='random',
#         n_init=10, max_iter=300,
#         tol=1e-04, random_state=0
#     )
#     km.fit(tfidf_matrix)
#     distortions.append(km.inertia_)

# # plot
# plt.plot(range(1, 20), distortions, marker='o')
# plt.xlabel('Number of clusters')
# plt.ylabel('Distortion')
# plt.show()

In [18]:
# import joblib

# Uncomment the below line to save your model 
#joblib.dump(km,  'doc_cluster.pkl')

# Uncomment the below line to load your saved model 
#km = joblib.load('doc_cluster.pkl')
#clusters = km.labels_.tolist()
# print("Clusters: {}".format(clusters))

In [14]:
data["cluster"] = clusters

print("Number of movies per cluster: \n{}".format(data["cluster"].value_counts()))

Number of movies per cluster: 
1    82932
6    13276
4    11693
5     9462
0     8415
7     7362
8     6409
3     5677
9     2339
2     2258
Name: cluster, dtype: int64


In [20]:
# data.groupby("cluster")["category"].value_counts()

## Researching clusters

In [15]:
print("Top terms per cluster:\n")

# Sort cluster centers by proximity to centroid.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

# Helper function
def getClusterWords(cluster, n=15):
    words = []
    for ind in order_centroids[cluster, :n]: # Print 6 words per cluster
        print(terms[ind])
        words.append(vocab_frame.loc[terms[ind].split(' '),].values.tolist()[0][0])
    return ", ".join(words)

for i in range(num_clusters):
    print("-"*50)
    print(f"Cluster {i}")
    
    print(getClusterWords(i))
    
    t = data[data["cluster"] == i]
    display(t)
    # Note that this does NOT add to 1, as one tweet has multiple categories
    display((t.drop(columns=["text", "cluster"]).sum() / len(t)).round(3))
    
#     break

    print("\n")

Top terms per cluster:

--------------------------------------------------
Cluster 0
retard
fuck retard
fuck
think
just
look
mental
peopl
went
shit
whi
say
trump
like
actual
retarded, fuck, fuck, thinking, just, look, mentally, people, went, shit, why, says, trump, like, actually


,text,Homophobe,NotHate,OtherHate,Racist,Religion,Sexist,cluster
1,horses are retarded,False,False,True,False,False,False,0
49,"Jeffery really said ""' take card from retard""",False,True,True,False,False,False,0
58,"Broke monitor because ’ fucking retard, now...",False,True,False,False,False,False,0
66,Ohhhh you retarded RETARDED,False,False,True,False,False,False,0
75,Battlefield arrivera finalement retard !,False,True,True,False,False,False,0
...,...,...,...,...,...,...,...,...
149796,"#PUBGMOBILE You went full retard man, never f...",False,True,True,False,False,False,0
149802,When she rolled her sleeves knew this interv...,False,False,True,True,False,False,0
149814,Ann shut you mentally retarded lunatic!,False,True,True,False,False,False,0
149817,"Thanks ""— retarded"" For Following !!!",False,True,False,False,False,True,0


Homophobe    0.033
NotHate      0.857
OtherHate    0.780
Racist       0.102
Religion     0.022
Sexist       0.049
dtype: float64



--------------------------------------------------
Cluster 1
nigga
nigger
got
redneck
fuck
ass
surrend
just
said
shit
real
hillbilli
say
don
sjw
nigga, nigger, got, redneck, fuck, ass, surrender, just, said, shit, real, hillbilly, says, don, sjw


,text,Homophobe,NotHate,OtherHate,Racist,Religion,Sexist,cluster
2,“NIGGA MOMMA YOUNGBOY SPITTING REAL SHIT NI...,False,True,False,False,False,False,1
3,xxSuGVNGxx: ran into this HOLY NIGGA TODAY 😭...,False,True,False,True,False,False,1
4,“EVERYbody calling you Nigger now!”,False,True,False,True,False,False,1
5,“ real ass bitch give fuck boutta nigga”,False,True,False,False,False,False,1
10,.... you turn twitter for instead beating ...,False,True,False,False,False,False,1
...,...,...,...,...,...,...,...,...
149815,know ’ too much real nigga ever danger ...,False,True,False,False,False,False,1
149816,Good luck nigga 🤘🏾,False,True,False,True,False,False,1
149820,nigga big shitty,False,True,False,True,False,False,1
149821,did she just say “ nigga” Rich? &amp; she sai...,False,True,False,True,False,False,1


...

,text,Homophobe,NotHate,OtherHate,Racist,Religion,Sexist,cluster
23,carol really said fuck yall ’ dyke and ’ here...,False,True,True,False,False,False,8
68,BIG DYKE ENERGY RIGHT THERE,True,True,False,False,False,False,8
76,cos theres strange brooding women dyke ener...,False,True,False,False,False,True,8
114,Jason Van Dyke' partner testifies murder trial,False,True,False,True,False,False,8
115,yes they can say dyke what gonna about huh,False,True,False,False,False,False,8
...,...,...,...,...,...,...,...,...
149681,Arlin Horton actually living life Dick Van D...,False,True,True,False,False,False,8
149751,Dyke New-Harford Pella Christian. Game . . #B...,False,True,False,False,False,False,8
149767,Protestors stopped from walking sidewalk outsi...,False,True,True,False,False,False,8
149780,’ gay and dyke and love food and this bodysu...,True,True,False,False,False,False,8


Homophobe    0.480
NotHate      0.915
OtherHate    0.147
Racist       0.102
Religion     0.017
Sexist       0.120
dtype: float64



--------------------------------------------------
Cluster 9
buildthewal
maga
border
wall
buildthewal maga
trump
presid
america
thank
amp
vote
stop
come
need
day
buildthewall, maga, border, wall, buildthewall, trump, president, america, thank, amp, vote, stop, coming, need, day


,text,Homophobe,NotHate,OtherHate,Racist,Religion,Sexist,cluster
145,This truth! #MAGA #KAG #BenghaziNeverForget #...,False,True,True,False,False,False,9
209,#BuildTheWall - Not President - Not Resident,False,True,False,True,False,False,9
212,"you #BuildTheWall, would good start you ...",False,True,False,False,False,False,9
259,More fake news!!!!!! Liar and enemy the peo...,False,True,True,True,False,False,9
276,THIS NOT GAME THE DIFFERENCE #MAGA #WalkA...,False,True,True,False,False,False,9
...,...,...,...,...,...,...,...,...
149490,"love you, . President! #MAGA #BuildTheWall #...",False,True,True,False,False,False,9
149647,#VoteRepublican STOP INVASION and #BuildTheW...,False,True,False,True,False,False,9
149690,#BuildTheWall #-VERIFY #PunishEmployersOfIlle...,True,True,False,True,False,False,9
149713,’ everywhere #RedWave #JobsNotMobs #Build...,False,True,False,False,False,False,9


Homophobe    0.022
NotHate      0.964
OtherHate    0.329
Racist       0.377
Religion     0.030
Sexist       0.021
dtype: float64